<a href="https://colab.research.google.com/github/hevertongomes/API-OMDb-JavaScript/blob/master/Atividade03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Atividade sobre mineração de regras de associação


> Está tarefa é voltada para análise de compras de veículos em 2021 baseado na tabela Fipe (Fundação Instituto de Pesquisas Econômicas).

#### Informações sobre o dataset escolhido:

> Link para base no kaggle: https://www.kaggle.com/victorsoeiro/brazilian-vehicle-prices-july-2021-fipe

In [52]:
import pandas as pd
import numpy as np

champs = pd.read_csv('https://raw.githubusercontent.com/ashouy/Aprendizado-de-Maquina/main/bases/tabela_fipe.csv')
champs.head()


,Marca,Modelo,Valor,Ano,Combustível,Automático
0,Acura,Integra GS 1.8,11165.0,1992,Gasolina,False
1,Acura,Integra GS 1.8,10167.0,1991,Gasolina,False
2,Acura,Legend 3.2/3.5,25574.0,1998,Gasolina,False
3,Acura,Legend 3.2/3.5,22561.0,1997,Gasolina,False
4,Acura,Legend 3.2/3.5,21388.0,1996,Gasolina,False


## Tratamento

In [53]:
# Renomenar colunas e diminuir tamanho da base para testes
champs.rename(columns={'Combustível':'Combustivel', 'Automático':'Cambio'}, inplace=True)
cars_fipe = champs.head(10000)

In [54]:
# Melhorando apresentação da coluna cambio para resultados
def convert_cambio_to_string(cambio):
  if cambio == False:
    return 'Manual'
  else:
    return 'Automático'

convert = cars_fipe['Cambio'].apply(convert_cambio_to_string)
convert.head()

0    Manual
1    Manual
2    Manual
3    Manual
4    Manual
Name: Cambio, dtype: object

In [55]:
cars_fipe['Cambio'] = convert
cars_fipe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Marca,Modelo,Valor,Ano,Combustivel,Cambio
0,Acura,Integra GS 1.8,11165.0,1992,Gasolina,Manual
1,Acura,Integra GS 1.8,10167.0,1991,Gasolina,Manual
2,Acura,Legend 3.2/3.5,25574.0,1998,Gasolina,Manual
3,Acura,Legend 3.2/3.5,22561.0,1997,Gasolina,Manual
4,Acura,Legend 3.2/3.5,21388.0,1996,Gasolina,Manual


In [56]:
cars_fipe.shape
# 171351 transações ferente a veículos da tabela fipe 

(10000, 6)

In [57]:
#Transforma o dataframe em uma lista de listas (cada lista é uma transação)
transactions = []
for i in range(0, cars_fipe.shape[0]):
    transactions.append([str(cars_fipe.values[i, j]) for j in range(0, 6)])

print(transactions[1])

['Acura', 'Integra GS 1.8', '10167.0', '1991', 'Gasolina', 'Manual']


In [58]:

#remove os nan das listas
for i in range(len(transactions)):
  transactions[i] = [item for item in transactions[i] if str(item) != 'nan']  

print(transactions[1])

['Acura', 'Integra GS 1.8', '10167.0', '1991', 'Gasolina', 'Manual']


## Aplicação do algoritmo apriori

In [59]:
!pip install apyori


In [61]:
from apyori import apriori
rules = apriori(transactions, min_support = 0.0045, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [62]:
association_rules = list(rules) #transforma resultado em uma lista

print(association_rules[0])

RelationRecord(items=frozenset({'2020', 'Automático'}), support=0.0112, ordered_statistics=[OrderedStatistic(items_base=frozenset({'2020'}), items_add=frozenset({'Automático'}), confidence=0.4132841328413284, lift=3.2465367858706085)])


In [50]:

#Mostra cada regra, com as informações de suporte, confiança e lift
for item in association_rules:
  # first index of the inner list
  # Contains base item and add item
  pair = item[0] 
  items = [x for x in pair]
  print("Rule: " + items[0] + " -> " + items[1])

  #second index of the inner list
  print("Support: " + str(item[1]))

  #third index of the list located at 0th
  #of the third index of the inner list

  print("Confidence: " + str(item[2][0][2]))
  print("Lift: " + str(item[2][0][3]))
  print("=====================================")

Rule: 2020 -> Automático
Support: 0.0112
Confidence: 0.4132841328413284
Lift: 3.2465367858706085
Rule: 2021 -> Automático
Support: 0.0095
Confidence: 0.38461538461538464
Lift: 3.021330593993595
Rule: Agrale -> Diesel
Support: 0.0076
Confidence: 1.0
Lift: 10.626992561105206
Rule: Caoa Chery -> Automático
Support: 0.0057
Confidence: 0.4488188976377953
Lift: 3.5256786931484316
Rule: Citroën -> Automático
Support: 0.0298
Confidence: 0.23409269442262373
Lift: 3.0801670318766283
Rule: Zero KM -> Automático
Support: 0.0122
Confidence: 0.40531561461794025
Lift: 3.183940413338101
Rule: 2020 -> Gasolina
Support: 0.0101
Confidence: 0.3726937269372694
Lift: 3.0623970989093623
Rule: 2021 -> Gasolina
Support: 0.0085
Confidence: 0.4009433962264151
Lift: 3.1495946286442664
Rule: Manual -> Agrale
Support: 0.0076
Confidence: 1.0
Lift: 11.299435028248588
Rule: Caoa Chery -> Gasolina
Support: 0.0057
Confidence: 0.4488188976377953
Lift: 3.687912059472435
Rule: Citroën -> Gasolina
Support: 0.0298
Confidence